In [ ]:
!pip install vllm transformers torch

In [ ]:
# セル 1: Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import time
from datetime import datetime

In [ ]:
import os
import json
import logging
from tqdm import tqdm
from vllm import LLM, SamplingParams

# 設定変数
NUM_SAMPLES = 20000  # 生成するサンプル数
OUTPUT_PATH = "/content/drive/MyDrive/synthesized_dataset.json"  # 出力ファイルパス
MODEL_PATH = "llm-jp/llm-jp-3-13b-instruct"  # モデルパス
BACKUP_INTERVAL = 100  # バックアップを作成する間隔
BATCH_SIZE = 5  # バッチサイズ
SAVE_INTERVAL = 10  # データセット保存間隔（バッチ単位）

# モデルの初期化
llm = LLM(
    model=MODEL_PATH,
    dtype="auto",
    trust_remote_code=True,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.8
)
tokenizer = llm.get_tokenizer()

DEFAULT_SYSTEM_PROMPT = "以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。"

INFO 10-12 17:21:51 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='llm-jp/llm-jp-3-13b-instruct', speculative_config=None, tokenizer='llm-jp/llm-jp-3-13b-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=llm-jp/llm-jp-3-13b-instruct, use_v2_block_manager=False, num_scheduler_steps=1, multi_step_stream_

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 10-12 17:22:01 model_runner.py:1025] Loading model weights took 25.5730 GB
INFO 10-12 17:22:03 gpu_executor.py:122] # GPU blocks: 394, # CPU blocks: 327
INFO 10-12 17:22:05 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-12 17:22:05 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-12 17:22:32 model_runner.py:1456] Graph capturing finished in 26 secs.


In [ ]:
def generate_with_chat_template_batch(chats: list, llm, max_new_tokens: int = 100, temperature: float = 0.7):
    try:
        prompts = [
            tokenizer.apply_chat_template(
                conversation=chat,
                add_generation_prompt=True,
                tokenize=False
            )
            for chat in chats
        ]

        sampling_params = SamplingParams(
            temperature=temperature,
            top_p=0.95,
            max_tokens=max_new_tokens,
            repetition_penalty=1.1,
        )

        outputs = llm.generate(prompts, sampling_params)

        results = []
        for output in outputs:
            if output.outputs:
                results.append(output.outputs[0].text.strip())
            else:
                print("警告: 出力が生成されませんでした。")
                results.append("")
        return results
    except Exception as e:
        print(f"エラー: generate_with_chat_template_batch で例外が発生しました: {e}")
        return [""] * len(chats)

def generate_instruction_batch(llm, batch_size: int):
    chats = [
        [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": "日本語で，エージェントに対する指示文を作成してください．\nこの時生成した指示文以外は生成しないでください．"
            }
        ]
        for _ in range(batch_size)
    ]
    instructions = generate_with_chat_template_batch(chats, llm, max_new_tokens=128, temperature=0.6)
    return instructions

def generate_cot_response_batch(instructions: list, llm, batch_size: int):
    prompts = [
        f"{instruction}\n\nこの指示を達成するための論理的な推論手順を段階的に説明し、最終的な回答を導いてください．"
        for instruction in instructions
    ]
    chats = [
        [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
        for prompt in prompts
    ]
    cot_responses = generate_with_chat_template_batch(chats, llm, max_new_tokens=256, temperature=0.7)
    return cot_responses

def refine_response_batch(instructions: list, cot_responses: list, llm, batch_size: int):
    chats = [
        [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": instruction
            },
            {
                "role": "assistant",
                "content": cot_response
            },
            {
                "role": "user",
                "content": "回答の改善すべき点を修正して，より完全な回答を導いてください，"
            }
        ]
        for instruction, cot_response in zip(instructions, cot_responses)
    ]
    refined_responses = generate_with_chat_template_batch(chats, llm, max_new_tokens=256, temperature=0.7)
    return refined_responses

def create_backup(dataset, output_path):
    backup_path = f"{output_path}.bak.{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    try:
        with open(backup_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)
        print(f"バックアップを作成しました: {backup_path}")
    except Exception as e:
        print(f"エラー: バックアップの作成中に例外が発生しました: {e}")

def synthesize_dataset_with_resume(num_samples: int, llm, output_path: str, backup_interval: int, batch_size: int, save_interval: int):
    if os.path.exists(output_path):
        print(f"既存のデータセットをロードしています: {output_path}")
        try:
            with open(output_path, "r", encoding="utf-8") as f:
                dataset = json.load(f)
        except Exception as e:
            print(f"エラー: データセットのロード中に例外が発生しました: {e}")
            dataset = []
    else:
        dataset = []

    start_idx = len(dataset)
    print(f"開始位置: {start_idx} / {num_samples}")

    start_time = time.time()

    for batch_num, i in enumerate(tqdm(range(start_idx, num_samples, batch_size), desc="サンプル生成中"), start=1):
        current_batch_size = min(batch_size, num_samples - i)
        try:
            # バッチサイズに応じて生成
            instructions = generate_instruction_batch(llm, current_batch_size)
            # チェック
            if not any(instructions):
                print(f"警告: バッチ {batch_num} ({i}) で指示の生成に失敗しました")
                instructions = [""] * current_batch_size

            cot_responses = generate_cot_response_batch(instructions, llm, current_batch_size)
            if not any(cot_responses):
                print(f"警告: バッチ {batch_num} ({i}) でCoT応答の生成に失敗しました")
                cot_responses = [""] * current_batch_size

            refined_responses = refine_response_batch(instructions, cot_responses, llm, current_batch_size)
            if not any(refined_responses):
                print(f"警告: バッチ {batch_num} ({i}) で洗練された応答の生成に失敗しました")
                refined_responses = [""] * current_batch_size

            # データセットに追加
            for instr, cot, refined in zip(instructions, cot_responses, refined_responses):
                if instr and cot and refined:
                    dataset.append({
                        "instruction": instr,
                        "cot_response": cot,
                        "response": refined
                    })
                else:
                    print(f"警告: バッチ {batch_num} ({i}) の一部で不完全なデータが生成されました")

            # データセット保存
            if batch_num % save_interval == 0:
                try:
                    with open(output_path, "w", encoding="utf-8") as f:
                        json.dump(dataset, f, ensure_ascii=False, indent=4)
                    print(f"データセットを保存しました: {output_path} (バッチ {batch_num})")
                except Exception as e:
                    print(f"エラー: データセットの保存中に例外が発生しました: {e}")

            # バックアップ作成
            if batch_num % backup_interval == 0:
                create_backup(dataset, output_path)

        except Exception as e:
            print(f"エラー: バッチ {batch_num} ({i}) でサンプル生成中に例外が発生しました: {e}")
            continue

    end_time = time.time()
    total_time = end_time - start_time
    processed_samples = len(dataset) - start_idx
    average_time = total_time / processed_samples if processed_samples > 0 else 0
    print(f"合計実行時間: {total_time:.2f} 秒")
    print(f"1サンプルあたりの平均時間: {average_time:.2f} 秒")

    # 最終的なデータセットを保存
    try:
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)
        print(f"最終的なデータセットを保存しました: {output_path}")
    except Exception as e:
        print(f"エラー: 最終データセットの保存中に例外が発生しました: {e}")


In [ ]:
if __name__ == "__main__":
    synthesize_dataset_with_resume(
        num_samples=NUM_SAMPLES,
        llm=llm,
        output_path=OUTPUT_PATH,
        backup_interval=BACKUP_INTERVAL,
        batch_size=BATCH_SIZE,
        save_interval=SAVE_INTERVAL
    )

既存のデータセットをロードしています: /content/drive/MyDrive/synthesized_dataset.json
開始位置: 9800 / 20000


Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  4.44it/s, est. speed input: 257.66 toks/s, output: 89.73 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.28s/it, est. speed input: 61.00 toks/s, output: 93.14 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  4.49it/s, est. speed input: 260.33 toks/s, output: 124.77 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it, est. speed input: 66.30 toks/s, output: 180.35 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:03<00:00,  1.59it/s, est. speed input: 92.28 toks/s, output: 73.83 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.30s/it, est. speed input: 80.35 toks/s, output: 183.90 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:02<00:00,  2.10it/s, est. speed input: 121.98 toks/s, output: 91.69 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it, est. speed input: 78.75 toks/s, output: 157.80 toks/s]

Processed prompts: 100%|█████████

データセットを保存しました: /content/drive/MyDrive/synthesized_dataset.json (バッチ 10)



Processed prompts: 100%|██████████| 5/5 [00:03<00:00,  1.58it/s, est. speed input: 91.54 toks/s, output: 97.85 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it, est. speed input: 92.83 toks/s, output: 161.42 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  2.70it/s, est. speed input: 156.76 toks/s, output: 80.00 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.30s/it, est. speed input: 67.35 toks/s, output: 193.28 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:00<00:00,  5.29it/s, est. speed input: 307.74 toks/s, output: 143.25 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.31s/it, est. speed input: 64.93 toks/s, output: 195.56 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:00<00:00,  5.42it/s, est. speed input: 315.14 toks/s, output: 70.63 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.27s/it, est. speed input: 55.79 toks/s, output: 127.77 toks/s]

Processed prompts: 100%|███████

データセットを保存しました: /content/drive/MyDrive/synthesized_dataset.json (バッチ 20)


...


Processed prompts: 100%|██████████| 5/5 [00:03<00:00,  1.60it/s, est. speed input: 92.73 toks/s, output: 88.58 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it, est. speed input: 87.86 toks/s, output: 198.00 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  3.86it/s, est. speed input: 224.13 toks/s, output: 140.66 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it, est. speed input: 73.24 toks/s, output: 197.37 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:00<00:00,  5.26it/s, est. speed input: 304.95 toks/s, output: 108.31 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.27s/it, est. speed input: 61.75 toks/s, output: 143.41 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:02<00:00,  1.86it/s, est. speed input: 108.29 toks/s, output: 96.72 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it, est. speed input: 84.83 toks/s, output: 197.79 toks/s]

Processed prompts: 100%|██████

データセットを保存しました: /content/drive/MyDrive/synthesized_dataset.json (バッチ 2030)



Processed prompts: 100%|██████████| 5/5 [00:00<00:00,  5.46it/s, est. speed input: 318.13 toks/s, output: 130.54 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.28s/it, est. speed input: 63.95 toks/s, output: 164.03 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  4.98it/s, est. speed input: 289.12 toks/s, output: 125.62 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.26s/it, est. speed input: 66.25 toks/s, output: 112.27 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  4.84it/s, est. speed input: 280.98 toks/s, output: 129.83 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.28s/it, est. speed input: 66.34 toks/s, output: 173.98 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  2.75it/s, est. speed input: 159.76 toks/s, output: 90.90 toks/s]

Processed prompts: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it, est. speed input: 70.58 toks/s, output: 198.55 toks/s]

Processed prompts: 100%|████

データセットを保存しました: /content/drive/MyDrive/synthesized_dataset.json (バッチ 2040)
合計実行時間: 30563.24 秒
1サンプルあたりの平均時間: 3.00 秒


最終的なデータセットを保存しました: /content/drive/MyDrive/synthesized_dataset.json


In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-3-13b-instruct")
# model = AutoModelForCausalLM.from_pretrained("llm-jp/llm-jp-3-13b-instruct", device_map="auto", torch_dtype=torch.bfloat16)
# chat = [
#     {"role": "system", "content": "以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。"},
#     {"role": "user", "content": "自然言語処理とは何か"},
# ]
# tokenized_input = tokenizer.apply_chat_template(chat, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(model.device)
# with torch.no_grad():
#     output = model.generate(
#         tokenized_input,
#         max_new_tokens=100,
#         do_sample=True,
#         top_p=0.95,
#         temperature=0.7,
#         repetition_penalty=1.05,
#     )[0]
# print(tokenizer.decode(output))